<h2>Código: generar dataset con missing (MCAR, MAR, MNAR) + CSV</h2>

In [4]:
import numpy as np
import pandas as pd

def generar_dataset_missing_por_tecnica(n=400, seed=42):
    rng = np.random.default_rng(seed)

    # Variables auxiliares para justificar decisiones
    grupo = rng.choice(["A", "B", "C"], size=n, p=[0.5, 0.3, 0.2])
    canal = rng.choice(["web", "tienda", "app"], size=n, p=[0.5, 0.25, 0.25])
    fecha = pd.to_datetime("2025-01-01") + pd.to_timedelta(rng.integers(0, 120, size=n), unit="D")

    # Base numérica correlacionada (para KNN / Iterative)
    x1 = rng.normal(0, 1, n)
    x2 = 0.7 * x1 + rng.normal(0, 0.5, n)
    x3 = 0.5 * x1 - 0.2 * x2 + rng.normal(0, 0.7, n)

    # Target (para ver si el missing podría estar sesgando)
    score = 0.9*x1 + 0.4*(canal=="app") - 0.3*(canal=="tienda") + rng.normal(0, 0.6, n)
    prob = 1/(1+np.exp(-score))
    target = (prob > 0.55).astype(int)

    df = pd.DataFrame({
        "fecha": fecha,
        "grupo": grupo,
        "canal": canal,
        "x1": x1, "x2": x2, "x3": x3,
        "target": target
    }).sort_values("fecha").reset_index(drop=True)

    # ---------------------------
    # Columnas “por técnica”
    # ---------------------------

    # (A) Drop rows: pocos missing MCAR (5%)
    df["num_drop_rows"] = 10 + 2*df["x1"] + rng.normal(0, 1, n)
    mask = rng.random(n) < 0.05
    df.loc[mask, "num_drop_rows"] = np.nan

    # (B) Drop columns: muchísimos missing (85%)
    df["col_drop_cols"] = rng.choice(["ok", "valor"], size=n).astype(object)
    mask = rng.random(n) < 0.85
    df.loc[mask, "col_drop_cols"] = np.nan

    # (C) Imputación media: distribución más “simétrica”
    df["num_mean"] = rng.normal(50, 10, n)
    mask = rng.random(n) < 0.15
    df.loc[mask, "num_mean"] = np.nan

    # (D) Imputación mediana: distribución sesgada + outliers
    df["num_median"] = rng.lognormal(mean=3.4, sigma=0.6, size=n)  # sesgada
    # outliers
    out_idx = rng.choice(np.arange(n), size=5, replace=False)
    df.loc[out_idx, "num_median"] *= 8
    mask = rng.random(n) < 0.15
    df.loc[mask, "num_median"] = np.nan

    # (E) Moda: categórica con pocos niveles
    df["cat_mode"] = rng.choice(["bajo", "medio", "alto"], size=n, p=[0.6, 0.3, 0.1]).astype(object)
    mask = rng.random(n) < 0.12
    df.loc[mask, "cat_mode"] = np.nan

    # (F) Constante: “desconocido” (missing con significado)
    df["cat_constant"] = rng.choice(["si", "no"], size=n, p=[0.7, 0.3]).astype(object)
    mask = rng.random(n) < 0.10
    df.loc[mask, "cat_constant"] = np.nan

    # (G) Por grupos: numérica depende de grupo (MAR)
    base = {"A": 100, "B": 150, "C": 200}
    df["num_group_median"] = df["grupo"].map(base).astype(float) + rng.normal(0, 12, n)
    # missing más frecuente en grupo C (MAR)
    mask = (df["grupo"] == "C") & (rng.random(n) < 0.35)
    df.loc[mask, "num_group_median"] = np.nan

    # (H) Flag + imputación: missing MNAR-like (falta más cuando es “alto”)
    df["num_flag"] = 30 + 6*df["x1"] + rng.normal(0, 2, n)
    thr = np.nanpercentile(df["num_flag"], 85)
    mask = (df["num_flag"] > thr) & (rng.random(n) < 0.45)
    df.loc[mask, "num_flag"] = np.nan

    # (I) KNN: columna correlacionada con x1,x2,x3
    df["num_knn"] = 5 + 2.2*df["x1"] - 1.1*df["x2"] + 0.6*df["x3"] + rng.normal(0, 0.6, n)
    mask = rng.random(n) < 0.18
    df.loc[mask, "num_knn"] = np.nan

    # (J) Iterative: otra columna correlacionada
    df["num_iterative"] = -2 + 1.7*df["x1"] + 0.4*df["x2"] + rng.normal(0, 0.7, n)
    mask = rng.random(n) < 0.18
    df.loc[mask, "num_iterative"] = np.nan

    # (K) Temporal: serie con huecos (ffill/interpolate)
    trend = np.linspace(100, 130, n)
    df["num_time"] = trend + rng.normal(0, 1.2, n)
    # huecos en “rachas”
    for start in rng.choice(np.arange(20, n-30), size=3, replace=False):
        df.loc[start:start+7, "num_time"] = np.nan

    return df

df_demo = generar_dataset_missing_por_tecnica(n=400, seed=42)
df_demo.to_csv("dataset_missing_tipos.csv", index=False)

df_demo.head()


,fecha,grupo,canal,x1,x2,x3,target,num_drop_rows,col_drop_cols,num_mean,num_median,cat_mode,cat_constant,num_group_median,num_flag,num_knn,num_iterative,num_time
0,2025-01-01,A,web,-1.733209,-1.528288,0.349864,0,6.973098,NaN,51.977320,36.412459,alto,si,89.052183,19.210646,3.388717,-5.139317,99.904604
1,2025-01-01,A,web,-0.302620,-0.043633,0.068503,1,7.585144,NaN,57.030255,24.631497,medio,no,104.038531,28.135892,4.177061,NaN,98.429387
2,2025-01-01,B,app,1.836226,1.348656,0.466935,1,NaN,NaN,62.370505,27.285677,bajo,si,164.729416,39.891790,7.858057,NaN,100.956497
3,2025-01-01,A,app,-0.218029,-0.292921,-0.691018,0,11.836934,NaN,NaN,25.151959,medio,no,112.469726,28.677732,NaN,-2.204822,99.356164
4,2025-01-01,B,web,0.700723,0.739866,-0.610250,1,12.041382,NaN,63.079156,115.019615,bajo,si,148.378644,35.628802,5.410144,-0.329172,99.872313


<h2>Código: generar dataset Outliers</h2>

In [1]:
import numpy as np
import pandas as pd

print("=== Generación de dataset para detección de outliers (Z-Score vs IQR) ===")

np.random.seed(42)

# -----------------------------
# 1) Datos "normales" (la mayoría de empleados)
# -----------------------------
# Salarios típicos: media ~2200, desviación ~400
normal_salaries = np.random.normal(loc=2200, scale=400, size=180)

# Nos aseguramos de que no haya valores negativos ni absurdos
normal_salaries = np.clip(normal_salaries, 900, 4000)

# -----------------------------
# 2) Outliers bajos (posibles errores, becarios, media jornada mal registrada, etc.)
# -----------------------------
low_outliers = np.array([300, 450, 600, 700])

# -----------------------------
# 3) Outliers altos (directivos, bonus extremos, errores de carga, etc.)
# -----------------------------
high_outliers = np.array([7000, 9000, 12000, 20000, 35000])

# -----------------------------
# 4) Mezclamos todo
# -----------------------------
salaries = np.concatenate([normal_salaries, low_outliers, high_outliers])

# Creamos un ID de empleado
employee_ids = [f"E{str(i).zfill(4)}" for i in range(1, len(salaries) + 1)]

df = pd.DataFrame({
    "employee_id": employee_ids,
    "salary": salaries
})

# Barajamos filas para que los outliers no queden todos al final
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# -----------------------------
# 5) Guardamos a CSV
# -----------------------------
csv_path = "dataset_outliers.csv"
df.to_csv(csv_path, index=False)



=== Generación de dataset para detección de outliers (Z-Score vs IQR) ===


<h2>Dataset para Categóricas</h2>

In [2]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

rng = np.random.default_rng(42)

def generar_dataset_categoricas(n=240):
    # -------------------------
    # Variables categóricas
    # -------------------------
    canal = rng.choice(
        ["web", "app", "tienda", "email"],
        size=n,
        p=[0.45, 0.25, 0.20, 0.10]
    )

    grupo = rng.choice(
        ["A", "B", "C"],
        size=n,
        p=[0.40, 0.35, 0.25]
    )

    # Variable ordinal
    cat_mode = rng.choice(
        ["bajo", "medio", "alto"],
        size=n,
        p=[0.35, 0.45, 0.20]
    )

    ciudad = rng.choice(
        ["Madrid", "Barcelona", "Valencia", "Sevilla",
         "Bilbao", "Zaragoza", "Málaga", "Granada"],
        size=n,
        p=[0.22, 0.18, 0.14, 0.12, 0.10, 0.10, 0.08, 0.06]
    )

    df = pd.DataFrame({
        "canal": canal,
        "grupo": grupo,
        "cat_mode": cat_mode,
        "ciudad": ciudad
    })

    # -------------------------
    # Introducir NaN (para ejemplos de imputación)
    # -------------------------
    for col, p_nan in [
        ("canal", 0.03),
        ("grupo", 0.03),
        ("cat_mode", 0.03),
        ("ciudad", 0.05)
    ]:
        mask = rng.random(n) < p_nan
        df.loc[mask, col] = np.nan

    # -------------------------
    # Generar target con señal real
    # (para que Target Encoding tenga sentido)
    # -------------------------
    city_effect = {
        "Madrid": 0.12, "Barcelona": 0.10, "Valencia": 0.06,
        "Sevilla": 0.04, "Bilbao": 0.02, "Zaragoza": 0.03,
        "Málaga": 0.05, "Granada": 0.01
    }

    ce = np.array([city_effect.get(v, 0.0) for v in df["ciudad"].fillna("NA")])

    canal_effect = (
        np.where(df["canal"].fillna("web") == "app", 0.10, 0.0) +
        np.where(df["canal"].fillna("web") == "email", -0.03, 0.0)
    )

    grupo_effect = (
        np.where(df["grupo"].fillna("A") == "C", 0.08, 0.0) +
        np.where(df["grupo"].fillna("A") == "B", 0.03, 0.0)
    )

    mode_effect = (
        np.where(df["cat_mode"].fillna("medio") == "alto", 0.07, 0.0) +
        np.where(df["cat_mode"].fillna("medio") == "bajo", -0.03, 0.0)
    )

    base = 0.30
    p = np.clip(
        base + ce + canal_effect + grupo_effect + mode_effect +
        rng.normal(0, 0.03, size=n),
        0.02, 0.98
    )

    df["target"] = (rng.random(n) < p).astype(int)

    # -------------------------
    # Añadir ciudades raras (para Frequency / Target Encoding)
    # -------------------------
    rare_cities = ["Oviedo", "Cáceres", "Lugo", "Huesca", "Ceuta"]

    for i, city in enumerate(rare_cities):
        df = pd.concat([df, pd.DataFrame([{
            "canal": "web",
            "grupo": "A",
            "cat_mode": "medio",
            "ciudad": city,
            "target": i % 2
        }])], ignore_index=True)

    return df


# =========================================================
# GENERAR Y GUARDAR DATASET
# =========================================================
df = generar_dataset_categoricas()

df.to_csv("dataset_categoricas_demo.csv", index=False, encoding="utf-8")




<h2>Dataset Escalado </h2>

In [2]:
import numpy as np
import pandas as pd

def generar_dataset_escalado(n=600, seed=42):
    rng = np.random.default_rng(seed)

    # Variable "pequeña" (0-10 aprox)
    horas_estudio = np.clip(rng.normal(5, 2, n), 0, None)

    # Variable "media" (18-70)
    edad = rng.integers(18, 70, size=n)

    # Variable "grande" (ingresos con cola larga: miles, con outliers)
    ingresos = rng.lognormal(mean=7.5, sigma=0.6, size=n)  # ~ 1k-10k con cola
    ingresos = ingresos * 1.0
    # Metemos algunos outliers muy grandes
    out_idx = rng.choice(np.arange(n), size=max(5, n//60), replace=False)
    ingresos[out_idx] *= rng.integers(8, 20, size=len(out_idx))

    # Variable acotada [0,1] (porcentaje/ratio)
    tasa_conversion = rng.beta(2, 10, size=n)

    # Variable centrada alrededor de 0 con positivos/negativos (p.ej. variación)
    delta_consumo = rng.normal(0, 15, size=n)

    # Variable "conteo" con sesgo (visitas web)
    visitas = rng.poisson(lam=40, size=n)

    # Un "target" opcional para que luego puedas usar modelos si quieres (no obligatorio aquí)
    score = (
        0.08 * horas_estudio
        + 0.015 * (edad - 35)
        + 0.00008 * ingresos
        + 2.5 * tasa_conversion
        + 0.01 * visitas
        + rng.normal(0, 1.0, size=n)
    )
    prob = 1 / (1 + np.exp(-score))
    target = (prob > 0.5).astype(int)

    df = pd.DataFrame({
        "horas_estudio": horas_estudio,
        "edad": edad,
        "ingresos": ingresos,
        "tasa_conversion": tasa_conversion,
        "delta_consumo": delta_consumo,
        "visitas": visitas,
        "target": target
    })

    return df

df = generar_dataset_escalado(n=600, seed=42)
df.to_csv("dataset_escalado.csv", index=False)

df.head()


,horas_estudio,edad,ingresos,tasa_conversion,delta_consumo,visitas,target
0,5.609434,30,1850.384388,0.370079,5.927597,31,1
1,2.920032,39,1499.597776,0.368863,4.846249,48,1
2,6.500902,21,3393.454698,0.093902,-4.741873,34,1
3,6.881129,59,1205.186513,0.053585,-16.524327,35,1
4,1.097930,51,1077.726732,0.078078,9.987496,37,1


<h2>Dataset para Feature Engineering </h2>

In [4]:
import numpy as np
import pandas as pd

def generar_dataset_feature_engineering(n=800, seed=42):
    rng = np.random.default_rng(seed)

    # Identificadores de cliente para features por grupo
    customer_id = rng.integers(1000, 1100, size=n)  # 100 clientes aprox

    # Categorías
    canal = rng.choice(["web", "app", "tienda"], size=n, p=[0.5, 0.3, 0.2])
    ciudad = rng.choice([f"ciudad_{i:02d}" for i in range(1, 21)], size=n)

    # Fechas (simula eventos/ventas en el tiempo)
    start = np.datetime64("2025-01-01")
    fecha = start + rng.integers(0, 365, size=n).astype("timedelta64[D]")

    # Numéricas (con sesgo y outliers)
    ingresos = rng.lognormal(mean=7.5, sigma=0.6, size=n)  # cola larga
    out_idx = rng.choice(np.arange(n), size=max(8, n//70), replace=False)
    ingresos[out_idx] *= rng.integers(5, 15, size=len(out_idx))

    edad = rng.integers(18, 70, size=n)
    visitas = rng.poisson(lam=35, size=n) + 1  # evitar cero
    tiempo_web_min = np.clip(rng.normal(8, 5, size=n), 0, None)

    # Texto simple (para ejemplo de text features)
    productos = rng.choice(["laptop", "movil", "tablet", "auriculares"], size=n, p=[0.2, 0.35, 0.15, 0.3])
    sentimiento = rng.choice(["bueno", "regular", "malo"], size=n, p=[0.6, 0.25, 0.15])
    comentario = (productos + " " + sentimiento)

    # Target (ejemplo de clasificación)
    score = (
        0.7*(canal == "app").astype(int)
        + 0.25*(canal == "web").astype(int)
        + 0.002*ingresos
        + 0.015*(edad-35)
        + 0.03*visitas
        + 0.04*tiempo_web_min
        + 0.4*(sentimiento == "bueno").astype(int)
        - 0.4*(sentimiento == "malo").astype(int)
        + rng.normal(0, 1.0, size=n)
    )
    prob = 1/(1+np.exp(-score))
    target = (prob > 0.6).astype(int)

    df = pd.DataFrame({
        "customer_id": customer_id,
        "fecha": pd.to_datetime(fecha),
        "canal": canal,
        "ciudad": ciudad,
        "edad": edad,
        "ingresos": ingresos,
        "visitas": visitas,
        "tiempo_web_min": tiempo_web_min,
        "comentario": comentario,
        "target": target
    })
    return df

df = generar_dataset_feature_engineering()
df.to_csv("dataset_feature_engineering.csv", index=False)

df.head()


,customer_id,fecha,canal,ciudad,edad,ingresos,visitas,tiempo_web_min,comentario,target
0,1008,2025-12-03,web,ciudad_01,57,977.778002,29,6.502571,auriculares bueno,1
1,1077,2025-11-30,tienda,ciudad_12,53,3117.659086,50,4.516700,tablet bueno,1
2,1065,2025-10-08,app,ciudad_15,36,878.645967,33,10.142415,tablet bueno,1
3,1043,2025-02-21,web,ciudad_12,51,1522.608587,45,9.173231,laptop bueno,1
4,1043,2025-04-20,tienda,ciudad_08,67,1272.029602,46,0.046752,auriculares bueno,1


<h2>Dataset para Feature Selection. Filter.Varianza </h2>

In [8]:
import numpy as np
import pandas as pd

def generar_dataset_filter(n=800, seed=42):
    rng = np.random.default_rng(seed)

    edad = rng.integers(18, 70, size=n)
    ingresos = rng.lognormal(mean=7.4, sigma=0.6, size=n)
    visitas = rng.poisson(lam=30, size=n)
    tiempo_web = np.clip(rng.normal(10, 5, size=n), 0, None)

    # Variables poco informativas
    constante = np.ones(n)
    casi_constante = rng.choice([0, 1], size=n, p=[0.98, 0.02])
    ruido = rng.normal(0, 1, size=n)

    score = (
        0.02 * (edad - 40)
        + 0.0008 * ingresos
        + 0.05 * visitas
        + 0.08 * tiempo_web
        + rng.normal(0, 1.0, size=n)
    )
    prob = 1 / (1 + np.exp(-score))
    target = (prob > 0.5).astype(int)

    df = pd.DataFrame({
        "edad": edad,
        "ingresos": ingresos,
        "visitas": visitas,
        "tiempo_web": tiempo_web,
        "constante": constante,
        "casi_constante": casi_constante,
        "ruido": ruido,
        "target": target
    })
    return df


df = generar_dataset_filter()
df.to_csv("dataset_feature_filter.csv", index=False)


,edad,ingresos,visitas,tiempo_web,constante,casi_constante,ruido,target
0,22,2129.797031,38,11.275967,1.0,0,0.287381,1
1,58,2240.630785,29,8.011110,1.0,0,0.154792,1
2,52,1930.944437,41,11.861628,1.0,0,-2.745614,1
3,40,700.882650,25,14.178075,1.0,0,-0.906474,1
4,40,409.091128,24,9.430418,1.0,0,-0.050847,1


<h2>Dataset para Feature Selection. Filter. Correlación Pearson (lineal) </h2>

In [5]:
import numpy as np
import pandas as pd

np.random.seed(42)
n = 800

# -------------------------
# Variables NUMÉRICAS
# -------------------------

# Relación LINEAL fuerte con el target → Pearson alto
tiempo_web = np.random.normal(loc=10, scale=3, size=n)

# Relación MONÓTONA (no lineal) → Spearman alto, Pearson bajo
ingresos = np.random.lognormal(mean=7.3, sigma=0.8, size=n)

# Relación por UMBRAL (útil en clasificación)
edad = np.random.randint(18, 70, size=n)

# Ruido puro
ruido = np.random.normal(0, 1, size=n)

# -------------------------
# Variable CATEGÓRICA
# -------------------------

# Categoría con dependencia clara del target → χ² alto
canal = np.random.choice(
    ["web", "app", "tienda"],
    size=n,
    p=[0.45, 0.35, 0.20]
)

# -------------------------
# Target binario controlado
# -------------------------

score = (
    1.3 * tiempo_web                     # relación lineal
    + 0.04 * np.log(ingresos)             # relación monótona
    + np.where(edad > 45, 1.0, 0.0)       # salto por clase
    + np.where(canal == "app", 1.2, 0.0)  # dependencia categórica
    + np.random.normal(0, 1.5, size=n)
)

prob = 1 / (1 + np.exp(-score))
target = (prob > np.median(prob)).astype(int)

df = pd.DataFrame({
    "tiempo_web": tiempo_web,
    "ingresos": ingresos,
    "edad": edad,
    "ruido": ruido,
    "canal": canal,
    "target": target
})




df.to_csv("dataset_feature_correlacion.csv", index=False)



<h2>Dataset para Feature Selection. WRAPPER </h2>

In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)
n = 600

edad = np.random.randint(18, 70, size=n)
ingresos = np.random.lognormal(mean=7.4, sigma=0.6, size=n)
visitas = np.random.poisson(lam=30, size=n)
tiempo_web = np.random.normal(10, 4, size=n)

ruido1 = np.random.normal(0, 1, size=n)
ruido2 = np.random.normal(0, 1, size=n)

score = (
    0.03 * (edad - 40)
    + 0.0008 * ingresos
    + 0.06 * visitas
    + 0.09 * tiempo_web
    + np.random.normal(0, 1.2, size=n)
)

prob = 1 / (1 + np.exp(-score))
target = (prob > np.median(prob)).astype(int)

df = pd.DataFrame({
    "edad": edad,
    "ingresos": ingresos,
    "visitas": visitas,
    "tiempo_web": tiempo_web,
    "ruido1": ruido1,
    "ruido2": ruido2,
    "target": target
})

df.to_csv("dataset_feature_wrapper.csv", index=False)


<h2>Dataset para Feature Selection. EMBEDDED </h2>

In [3]:
import numpy as np
import pandas as pd

np.random.seed(42)
n = 800

# Variables informativas
edad = np.random.randint(18, 70, size=n)
ingresos = np.random.lognormal(mean=7.4, sigma=0.6, size=n)
visitas = np.random.poisson(lam=25, size=n)
tiempo_web = np.random.normal(10, 4, size=n)

# Variables redundantes (muy correlacionadas con informativas)
ingresos_dup = ingresos * 1.02 + np.random.normal(0, ingresos.std()*0.02, size=n)
visitas_dup = visitas + np.random.normal(0, 2, size=n)

# Ruido
ruido1 = np.random.normal(0, 1, size=n)
ruido2 = np.random.normal(0, 1, size=n)

# Señal para clasificación binaria
score = (
    0.04 * (edad - 40)
    + 0.0007 * ingresos
    + 0.08 * visitas
    + 0.12 * tiempo_web
    + np.random.normal(0, 1.3, size=n)
)

p = 1 / (1 + np.exp(-score))
target = (np.random.rand(n) < p).astype(int)

df = pd.DataFrame({
    "edad": edad,
    "ingresos": ingresos,
    "visitas": visitas,
    "tiempo_web": tiempo_web,
    "ingresos_dup": ingresos_dup,
    "visitas_dup": visitas_dup,
    "ruido1": ruido1,
    "ruido2": ruido2,
    "target": target
})


df.to_csv("dataset_feature_embedded.csv", index=False)


<h2>Dataset Reducción de la Dimensionalidad </h2>

In [3]:
import numpy as np
import pandas as pd

# Semilla para reproducibilidad
np.random.seed(42)
n = 800

# -----------------------------
# Variables informativas
# -----------------------------
edad = np.random.randint(18, 70, size=n)
ingresos = np.random.lognormal(mean=7.4, sigma=0.6, size=n)
visitas = np.random.poisson(lam=25, size=n)
tiempo_web = np.random.normal(10, 4, size=n)

# -----------------------------
# Variables redundantes
# (muy correlacionadas con las informativas)
# -----------------------------
ingresos_dup = ingresos * 1.02 + np.random.normal(0, ingresos.std() * 0.02, size=n)
visitas_dup = visitas + np.random.normal(0, 2, size=n)

# -----------------------------
# Variables de ruido
# -----------------------------
ruido1 = np.random.normal(0, 1, size=n)
ruido2 = np.random.normal(0, 1, size=n)

# -----------------------------
# Generación del target binario
# -----------------------------
score = (
    0.04 * (edad - 40)
    + 0.0007 * ingresos
    + 0.08 * visitas
    + 0.12 * tiempo_web
    + np.random.normal(0, 1.3, size=n)
)

p = 1 / (1 + np.exp(-score))
target = (np.random.rand(n) < p).astype(int)

# -----------------------------
# Dataset final
# -----------------------------
df = pd.DataFrame({
    "edad": edad,
    "ingresos": ingresos,
    "visitas": visitas,
    "tiempo_web": tiempo_web,
    "ingresos_dup": ingresos_dup,
    "visitas_dup": visitas_dup,
    "ruido1": ruido1,
    "ruido2": ruido2,
    "target": target
})

# Guardar a CSV
df.to_csv("dataset_reduccion_dimensionalidad.csv", index=False)


<h2>Dataset Balanceo de Clases. Undersampling </h2>

In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification


# ============================================================
# GENERACIÓN DEL DATASET DESBALANCEADO
# ============================================================
def generar_dataset_csv(
    ruta_csv="dataset_desbalanceado.csv",
    n_samples=200_000,
    n_features=30,
    imbalance_ratio=0.02,   # 2% clase minoritaria
    random_state=42
):
    """
    Genera un dataset binario desbalanceado y lo guarda en un CSV.
    La columna 'target' es la variable objetivo.
    """

    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=int(n_features * 0.4),
        n_redundant=int(n_features * 0.2),
        n_repeated=0,
        n_classes=2,
        weights=[1 - imbalance_ratio, imbalance_ratio],
        class_sep=1.0,
        flip_y=0.01,
        random_state=random_state
    )

    # Crear DataFrame
    columnas = [f"feature_{i+1}" for i in range(n_features)]
    df = pd.DataFrame(X, columns=columnas)
    df["target"] = y

    # Guardar CSV
    df.to_csv(ruta_csv, index=False)

    print(f"Dataset generado y guardado en: {ruta_csv}")
    print("Distribución de clases:")
    print(df["target"].value_counts(normalize=True))


if __name__ == "__main__":
    generar_dataset_csv(
        ruta_csv="dataset_desbalanceado.csv",
        n_samples=200_000,
        n_features=30,
        imbalance_ratio=0.02
    )


Dataset generado y guardado en: dataset_desbalanceado.csv
Distribución de clases:
target
0    0.97499
1    0.02501
Name: proportion, dtype: float64


<h2>Dataset para Feature Engineering</h2>

In [1]:
import numpy as np
import pandas as pd

# =========================
# 1) GENERAR DATASET EJEMPLO
# =========================
np.random.seed(42)

n_customers = 25
n_days = 60
records_per_day = 8  # número de compras por día (aprox)

customers = [f"C{str(i).zfill(3)}" for i in range(1, n_customers + 1)]
categories = ["electronics", "fashion", "home", "sports", "books"]

dates = pd.date_range("2024-01-01", periods=n_days, freq="D")

rows = []
order_id = 1

# Datos "de cliente" (se repiten en sus pedidos)
customer_profile = {
    c: {
        "age": np.random.randint(18, 65),
        "height_m": np.round(np.random.uniform(1.55, 1.95), 2),
        "weight_kg": np.round(np.random.uniform(50, 110), 1),
        "distance_km_base": np.random.uniform(1, 25)  # distancia típica al almacén
    }
    for c in customers
}

for d in dates:
    # número variable de pedidos cada día
    n_orders = max(2, int(np.random.normal(records_per_day, 2)))
    for _ in range(n_orders):
        cust = np.random.choice(customers)
        cat = np.random.choice(categories, p=[0.22, 0.25, 0.2, 0.18, 0.15])

        # Precios según categoría
        base_price = {
            "electronics": np.random.uniform(50, 600),
            "fashion": np.random.uniform(10, 120),
            "home": np.random.uniform(15, 200),
            "sports": np.random.uniform(12, 180),
            "books": np.random.uniform(5, 40),
        }[cat]

        unit_price = np.round(base_price * np.random.uniform(0.9, 1.15), 2)
        unit_cost = np.round(unit_price * np.random.uniform(0.55, 0.85), 2)  # coste < precio

        quantity = np.random.randint(1, 6)

        # Distancia (km) con algo de variación diaria
        distance_km = np.round(
            customer_profile[cust]["distance_km_base"] * np.random.uniform(0.7, 1.4),
            2
        )

        # Coste de envío simulado (no es "feature engineering", solo dato del dataset)
        shipping_cost = np.round(2.5 + 0.35 * distance_km + np.random.uniform(-0.5, 0.8), 2)
        shipping_cost = max(1.5, shipping_cost)

        rows.append({
            "order_id": f"O{order_id:05d}",
            "date": d,
            "customer_id": cust,
            "category": cat,
            "unit_price": unit_price,
            "unit_cost": unit_cost,
            "quantity": quantity,
            "distance_km": distance_km,
            "shipping_cost": shipping_cost,
            "age": customer_profile[cust]["age"],
            "height_m": customer_profile[cust]["height_m"],
            "weight_kg": customer_profile[cust]["weight_kg"],
        })
        order_id += 1

df = pd.DataFrame(rows)

# Guardar CSV
csv_path = "dataset_crear_variables.csv"
df.to_csv(csv_path, index=False)



<h2>Dataset Feature Importance</h2>

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

# Generamos un dataset con estructura controlada
X, y = make_classification(
    n_samples=2000,
    n_features=12,        # Total de variables
    n_informative=4,      # Realmente importantes
    n_redundant=3,        # Correlacionadas con las importantes
    n_repeated=0,
    n_clusters_per_class=2,
    flip_y=0.03,          # Un poco de ruido en las etiquetas
    class_sep=1.2,
    random_state=42
)

# Nombres de columnas más explicativos
feature_names = [
    "signal_1", "signal_2", "signal_3", "signal_4",      # informativas
    "redundant_1", "redundant_2", "redundant_3",          # correlacionadas
    "noise_1", "noise_2", "noise_3", "noise_4", "noise_5" # ruido puro
]

df = pd.DataFrame(X, columns=feature_names)
df["target"] = y

# Guardar CSV
csv_path = "dataset_feature_importance.csv"
df.to_csv(csv_path, index=False)
